# Upload video


In [1]:
from google.colab import files
uploaded = files.upload()

Saving machine_learning.mp4 to machine_learning.mp4


In [2]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=16fb3ae7b605f9e201c1fb396719fe0a2642438cf0c2d804ebe95194dc71bdae
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [3]:
!pip install transformers torch

In [4]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [5]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dc03da1f8a937c7d490ef54bd80f714ddceb2b2055d23483d0ef3c9484eba81f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [6]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


enter your GROQ_API_KEY below

In [7]:
GROQ_API_KEY="your_groq_api_key"
import os

# Set your API key here
os.environ['GROQ_API_KEY'] = "your_groq_api_key"

# Imports

In [8]:
import whisper
import os
from transformers import pipeline
from datetime import timedelta
from groq import Groq

In [9]:
import re
import torch
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# 1: function ot enable gpu if available

In [10]:
def enable_gpu():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print("GPU is available. Using GPU for transcription.")
    else:
        device = torch.device('cpu')
        print("GPU not available. Using CPU for transcription.")
    return device

Step 2: Load and transcribe the whole video using Whisper with GPU support

In [11]:
VIDEO_PATH = "/content/machine_learning.mp4"
def transcribe_video(video_path):
    device = enable_gpu()  # Enable GPU
    model = whisper.load_model("base").to(device)
    result = model.transcribe(video_path)
    print(result)
    return result

Step 3: Get start and end timestamps from user

In [12]:
def get_user_timestamps():
    start = input("Enter start timestamp (in format HH:MM:SS): ")
    end = input("Enter end timestamp (in format HH:MM:SS): ")
    return start, end

Step 4: Extract transcript between timestamps

In [13]:

def extract_transcript(transcription, start_time, end_time):
    # Convert timestamps to seconds
    start_seconds = sum(int(x) * 60 ** i for i, x in enumerate(reversed(start_time.split(':'))))
    end_seconds = sum(int(x) * 60 ** i for i, x in enumerate(reversed(end_time.split(':'))))

    extracted_text = ""
    continue_transcribing = False

    # Go through each transcription segment
    for segment in transcription['segments']:
        if start_seconds <= segment['start'] and (segment['end'] <= end_seconds or continue_transcribing):
            extracted_text += segment['text'] + " "

        # If the segment is within or past end_time, check for full stop to continue or stop
        if segment['end'] > end_seconds:
            continue_transcribing = True  # Start adding extra text until full stop

            # Append segment's text
            extracted_text += segment['text'] + " "

            # Check for the first full stop after end_time
            if re.search(r'[.!?]', segment['text']):
                # Truncate the text at the first full stop after end_time
                match = re.search(r'.*?[.!?]', segment['text'])
                if match:
                    extracted_text = extracted_text.rstrip() + match.group()  # Append only until the first full stop
                break  # Stop extracting after finding the full stop

    return extracted_text.strip()

Step 5: Summarize the extracted transcript using Groq

In [14]:

def summarize_with_groq(extracted_transcript, min_words=15, max_words=25):
    # Initialize Groq client with API key
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),  # Ensure API key is set in environment
    )

    # Creating a system prompt for summarization, using min_words and max_words in the prompt
    system_prompt = {
        "role": "system",
        "content": f"Only output the summary sentence of the following transcript with the number of words greater than {min_words} and less than {max_words} in one sentence."

    }

    # Adding the extracted transcript as user input
    user_prompt = {
        "role": "user",
        "content": extracted_transcript
    }

    # Make Groq request
    chat_completion = client.chat.completions.create(
        messages=[system_prompt, user_prompt],
        model="llama3-8b-8192",  # You can change the model if needed
    )

    summary = chat_completion.choices[0].message.content
    return summary

Step 7: Evaluation

Function to calculate ROUGE-L scores

In [15]:

def calculate_rouge_l(generated_summary, reference_summary):
    # Initialize ROUGE scorer with ROUGE-L metric
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    # Compute ROUGE-L between generated and reference summaries
    score = scorer.score(reference_summary, generated_summary)

    # Extract the precision, recall, and F1-score
    rouge_l_precision = score['rougeL'].precision
    rouge_l_recall = score['rougeL'].recall
    rouge_l_f1 = score['rougeL'].fmeasure

    # Return the scores
    return {
        "ROUGE-L Precision": rouge_l_precision,
        "ROUGE-L Recall": rouge_l_recall,
        "ROUGE-L F1-Score": rouge_l_f1
    }

Function to calculate BERTScore

In [16]:

def calculate_bert_score(generated_summary, reference_summary):
    # Use BERTScore with a pre-trained model (by default, it uses RoBERTa model)
    P, R, F1 = bert_score([generated_summary], [reference_summary], lang="en", verbose=True)

    # Return the precision, recall, and F1-score
    return {
        "BERTScore Precision": P.mean().item(),
        "BERTScore Recall": R.mean().item(),
        "BERTScore F1-Score": F1.mean().item()
    }

Main execution

In [17]:

def main():
    print("Transcribing video...")
    transcription = transcribe_video(VIDEO_PATH)

    start_time, end_time = get_user_timestamps()

    print("Extracting transcript between timestamps...")
    extracted_transcript = extract_transcript(transcription, start_time, end_time)

    print("Summarizing extracted transcript using Groq LLM...")
    summary = summarize_with_groq(extracted_transcript)

    print("\nExtracted Transcript:")
    print(extracted_transcript)

    print("\nSummary for PowerPoint slide:")
    print(summary)

    # Take input ideal summary
    ideal_summary = input("Enter the ideal summary: ")

    print("\nCalculating ROUGE-L scores...")
    rouge_l_scores = calculate_rouge_l(summary, ideal_summary)
    print(rouge_l_scores)

    print("\nCalculating BERTScore...")
    bert_scores = calculate_bert_score(summary, ideal_summary)
    print(bert_scores)

if __name__ == "__main__":
    main()

Transcribing video...
GPU not available. Using CPU for transcription.


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 131MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

{'text': " Sci-fi movies and books depict a future with sentinant machines capable of interacting with humans and performing tasks just like people would. What if we told you that this future is already a reality? All of this is already made possible by machine learning. Machine learning is a science of programming machines to think and act like humans without being specifically programmed to. We already use machine learning in our daily life without knowing it. E-mails Pam recognition, spellcheck, even the YouTube video recommendation which brought you here are implemented using machine learning. Machine learning uses algorithms to learn tasks. These algorithms are fed with data from which they learn to perform these tasks. This means that over time, as changes in data occur, we don't need to reprogram our application. Just let it find patterns and learn from the new data. Machine learning is a subset of artificial intelligence, which is a science concern with imparting human-like int

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.11 seconds, 0.90 sentences/sec
{'BERTScore Precision': 0.82513427734375, 'BERTScore Recall': 0.8953995704650879, 'BERTScore F1-Score': 0.8588321805000305}


# Results



Extracted Transcript:
Sci-fi movies and books depict a future with sentinant machines capable of  interacting with humans and performing tasks just like people would. What if we  told you that this future is already a reality? All of this is already made  possible by machine learning. Machine learning is a science of programming  machines to think and act like humans without being specifically programmed to.  We already use machine learning in our daily life without knowing it.  E-mails Pam recognition, spellcheck, even the YouTube video recommendation which  brought you here are implemented using machine learning. Machine learning uses  algorithms to learn tasks. These algorithms are fed with data from which they  learn to perform these tasks. This means that over time, as changes in data  occur, we don't need to reprogram our application. Just let it find patterns and  learn from the new data. Machine learning is a subset of artificial  intelligence, which is a science concern with imparting human-like  intelligence onto machines and creating machines which can sense reason, act,  and adapt. Deep learning is a sub-branch of machine learning, which is  inspired by the working of the human brain. Machine learning is leading us to a  future where machines can learn and think and has opened us a whole new  plethora of job opportunities. This was a brief intro to machine learning. If  you like this video, do give it a thumbs up and subscribe to the Simply  Learn channel for more content like this. Don't forget to hit the bell icon to Learn channel for more content like this.

Summary for PowerPoint slide:

"Machine learning is a science of programming machines to think and act like humans without being specifically programmed to."


Ideal summary:

Machine learning enables machines to learn and think like humans using algorithm to find patterns.

ROUGE-L F1-Score: 0.3018867924528302


BERTScore F1-Score: 0.8588321805000305